In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
from dateutil import parser
from functools import reduce

In [2]:
root = os.path.join(os.getcwd(), "data\\")
csv_files = {os.path.basename(p) : glob.glob(os.path.join(p, "*.csv")) for p, _, files in os.walk(root) if len(files) > 0}
datasets = {folder:{os.path.basename(f): pd.read_csv(f) for f in fs} for folder, fs in csv_files.items()}
ids_in_files = {folder:{f:np.unique(ds['Id'].values) for f, ds in dataset.items()} for folder, dataset in datasets.items()}
IDs = {f:reduce(np.intersect1d, [v for v in vs.values()]) for f, vs in ids_in_files.items()}
{f:len(ids) for f, ids in IDs.items()}

{'daily': 24, 'hourly': 33, 'minute': 24, 'seconds': 14}

In [3]:
date_granularity = {
    'daily':    '%Y-%m-%d 00:00:00',
    'hourly':   '%Y-%m-%d %H:00:00',
    'minute':   '%Y-%m-%d %H:%M:00',
    'seconds':  '%Y-%m-%d %H:%M:%S'
}

for f, ids in IDs.items():
    folder = os.getcwd() + '\\out\\' + f
    os.makedirs(folder, exist_ok=True) 
    for id in ids:
        df = pd.DataFrame(columns=['date'])
        dates = list(sorted(list(set(parser.parse(date).strftime(date_granularity[f]) for ds in datasets[f].values() for date in ds[ds['Id'] == id].iloc[:, 1]))))
        df['date'] = dates

        for ds in datasets[f].values():
            ds = ds[ds['Id'] == id].copy()
            ds.iloc[:, 1] = ds.iloc[:, 1].apply(lambda d:parser.parse(d).strftime(date_granularity[f]))
            ds = ds.drop(columns='Id')

            df = df.merge(ds, how='left', left_on='date', right_on=ds.keys()[0])
            if ds.keys()[0] != 'date':
                df = df.drop(columns=ds.keys()[0])
        df.to_csv(f'{folder}\\{id}.csv')